In [ ]:
import numpy as np
import tensorflow as tf
import math
from random import random
import random as rand

# parameters used
MODEL_PATH = 'model/Seq2Seqmodel_CRF.ckpt'

In [1]:

# Create a sequence classification instance.
def get_sequence(sequence_length):
    # Create a sequence of random numbers in between 0,1
#     X = np.random.rand(sequence_length)
    X = np.array([random() for _ in range(sequence_length)])
    # Calculate cut-off value to change class values.
    limit = sequence_length / 4.0
    # Determine the class outcome for each item in cumulative sequence 
    y = np.array([0 if x < limit else 1 for x in np.cumsum(X)])
    
    return X, y 
    ## ^ X = Sequence, y = sequence - both have samp dim

# create n examples with random sequence lengths between 5 and 15
def get_examples(n):
    X_list = []
    y_list = []
    sequence_length_list = []
    
    for _ in range(n):
#         sequence_length = np.random.randint(5, 16, 1)
        sequence_length = rand.randrange(start=5, stop=15)
        X, y = get_sequence(sequence_length)
        X_list.append(X)
        y_list.append(y)
        sequence_length_list.append(sequence_length)
        
    return X_list, y_list, sequence_length_list
    
# Tensorflow requires that all sentences(and all lables)
# inside the same batch have the same length, 
# so we have to pad the data(and labels) inside the batches (with 0`s, for exmaple)
def pad(sentence, max_length):
    pad_len = max_length - len(sentence)
    padding = np.zeros(pad_len)
    return np.concatenate((sentence, padding))

# Create input batches
def batch(data, labels, sequence_lengths, batch_size, input_size):
    n_batch = int(math.ceil(len(data) / batch_size)) #올림 
    index = 0 
    for _ in range(n_batch):
        batch_sequence_lengths = np.array(sequence_lengths[index: index + batch_size])
        
#         batch_length = np.array(tf.map_fn(max, batch_sequence_lengths, tf.int32))
        batch_length = np.array(max(batch_sequence_lengths)) # max length in batch 
        # pad data
        batch_data = np.array([pad(x, batch_length) for x in data[index: index + batch_size]])
        # pad labels
        batch_labels = np.array([pad(x, batch_length) for x in labels[index: index + batch_size]])
        
        index += batch_size
        
        # Reshape input data to be suitable for LSTMs.
        batch_data = batch_data.reshape(-1, batch_length, input_size)
        
        yield batch_data, batch_labels, batch_length, batch_sequence_lengths
        



/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch: 0, Accuracy: 62.92834890965732
Epoch: 0, Accuracy: 63.87096774193548
Epoch: 0, Accuracy: 58.53658536585366
Epoch: 0, Accuracy: 70.73170731707317
Epoch: 10, Accuracy: 91.90031152647975
Epoch: 10, Accuracy: 89.6774193548387
Epoch: 10, Accuracy: 91.15853658536585
Epoch: 10, Accuracy: 87.8048780487805
Epoch: 20, Accuracy: 94.0809968847352
Epoch: 20, Accuracy: 90.64516129032258
Epoch: 20, Accuracy: 93.59756097560975
Epoch: 20, Accuracy: 87.8048780487805
Epoch: 30, Accuracy: 95.95015576323988
Epoch: 30, Accuracy: 92.90322580645162
Epoch: 30, Accuracy: 95.1219512195122
Epoch: 30, Accuracy: 92.6829268292683
Epoch: 40, Accuracy: 94.70404984423676
Epoch: 40, Accuracy: 93.54838709677419
Epoch: 40, Accuracy: 95.42682926829268
Epoch: 40, Accuracy: 95.1219512195122
Epoch: 50, Accuracy: 95.32710280373831
Epoch: 50, Accuracy: 94.83870967741936
Epoch: 50, Accuracy: 94.8170731707317
Epoch: 50, Accuracy: 95.1219512195122
Epoch: 60, Accuracy: 96.26168224299066
Epoch: 60, Accuracy: 94.83870967741936

In [ ]:
# Generate train and test data. 
x_train, y_train, sequence_length_train = get_examples(100)
x_test, y_test, sequence_length_test = get_examples(30)

In [ ]:

#### Training the model #### 

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(training_epochs):
        for batch_data, batch_labels, batch_seq_len, batch_sequence_lengths in batch(x_train, y_train, sequence_length_train, batch_size, input_size):
            
            tf_viterbi_sequence, _ = sess.run([viterbi_sequence, train_op], 
                                             feed_dict={input_data: batch_data,
                                                       labels: batch_labels,
                                                       batch_sequence_length: batch_seq_len,
                                                       original_sequence_lengths: batch_sequence_lengths})
            # Show train accuracy 
            if i % 10 == 0 :
                # Create a mask to fix input lengths. 
                mask = (np.expand_dims(np.arange(batch_seq_len), axis=0) < np.expand_dims(batch_sequence_lengths, axis=1))
                total_labels = np.sum(batch_sequence_lengths)
                correct_labels = np.sum((batch_labels == tf_viterbi_sequence) * mask)
                accuracy = 100.0 * correct_labels / float(total_labels)
                print("Epoch: {}, Accuracy: {}".format(i, accuracy))
                
    saver.save(sess, MODEL_PATH)